In [1]:
import pickle
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

In [2]:
print (pd.__version__)
pd.show_versions()

0.24.2

INSTALLED VERSIONS
------------------
commit: None
python: 2.7.15.final.0
python-bits: 64
OS: Linux
OS-release: 4.4.0-179-generic
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: None.None

pandas: 0.24.2
pytest: 3.2.1
pip: 19.2.3
setuptools: 38.4.0
Cython: 0.26.1
numpy: 1.14.0
scipy: 1.0.0
pyarrow: None
xarray: None
IPython: 5.4.1
sphinx: 1.6.3
patsy: 0.5.1
dateutil: 2.6.1
pytz: 2017.2
blosc: None
bottleneck: 1.2.1
tables: 3.4.2
numexpr: 2.6.2
feather: None
matplotlib: 2.2.5
openpyxl: 2.4.8
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.2
lxml.etree: None
bs4: 4.6.0
html5lib: 0.999999999
sqlalchemy: 1.1.13
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None
gcsfs: None


In [3]:
base = 'ks5'
path_front = '/home/kazuki/^profilingdata/20160420_reanalyze/expression/'
path_rear = '_3map_gene_expression.csv'

file_path = path_front+base+path_rear
column_set= pd.MultiIndex.from_product([['ks5'],['reads','rpc','rpkm']])
df_base = DataFrame.from_csv(file_path)
df_base.columns = column_set

/home/kazuki/.pyenv/versions/anaconda2-5.0.1/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  import sys


In [4]:
files = ['ks' + str(x) for x in range(6,23)]
path_front = '/home/kazuki/^profilingdata/20160420_reanalyze/expression/'
path_rear = '_3map_gene_expression.csv'

for f in files:
    column_set= pd.MultiIndex.from_product([[f],['reads','rpc','rpkm']])
    df = DataFrame.from_csv(path_front+f+path_rear)
    df.columns = column_set
    df_base = pd.concat([df_base,df],axis=1)

/home/kazuki/.pyenv/versions/anaconda2-5.0.1/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  import sys


In [5]:
files = ['ks' + str(x) for x in range(5,23)]
path_front = '/home/kazuki/^profilingdata/20160420_reanalyze/expression_3UTR/'
path_rear = '_3map_gene_expression.csv'

for f in files:
    column_set= pd.MultiIndex.from_product([[f+'_3utr'],['reads','rpc','rpkm']])
    df = DataFrame.from_csv(path_front+f+path_rear)
    df.columns = column_set
    df_base = pd.concat([df_base,df],axis=1)

/home/kazuki/.pyenv/versions/anaconda2-5.0.1/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  import sys
/home/kazuki/.pyenv/versions/anaconda2-5.0.1/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [6]:
mRNA_files =  ['ks'+str(x) for x in range(17,23)]
Normal_files = ['ks'+str(x) for x in range(5,11)]
NaCl_files = ['ks'+str(x) for x in range(11,17)]

for normal,mRNA in zip(Normal_files,mRNA_files):
    df_base['TE_'+normal+'/'+mRNA]=df_base[normal,'rpkm']/df_base[mRNA,'rpkm']

for NaCl,mRNA in zip(NaCl_files,mRNA_files):
    df_base['TE_'+NaCl+'/'+mRNA]=df_base[NaCl,'rpkm']/df_base[mRNA,'rpkm']        

In [7]:
with open('/home/kazuki/^data_analysis/^Ecoli_Genome/Ecoli_Gene_TU.pickle','rb') as f:
    Gene_TU_dict = pickle.load(f)
    
Gene_TU_DataFrame = DataFrame.from_dict(Gene_TU_dict, orient='index')
Gene_TU_DataFrame.columns = ['strand','gene_type','gene_start','gene_end','TU','TU_start','TU_end','MorP']
df_base = pd.concat([df_base,Gene_TU_DataFrame],axis=1)

/home/kazuki/.pyenv/versions/anaconda2-5.0.1/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [8]:
# storeing once to make MultiIndex columns name to be one layer

df_base.index.name = 'gene'
df_base.reset_index(inplace=True)
CDS = df_base['gene_type'] == 'CDS'
df_base_CDS = df_base[CDS]

df_base_CDS.to_csv('rrf_CDS.csv') 
df_base_CDS = pd.read_csv('rrf_CDS.csv')
df_base_CDS = df_base_CDS.loc[:,~df_base_CDS.columns.str.contains('^Unnamed')]

In [9]:
new_column_names = []
for cn in df_base_CDS.columns:
    new_cn=cn
    if cn[0]=='(':
        cn_list = cn.split(',')
        new_cn = cn_list[0][2:-1]+'_'+cn_list[1][2:-2]
    if new_cn[-1]=='_':
        new_cn = new_cn[:-1]
    new_column_names.append(new_cn)
print len(new_column_names)

129


In [10]:
df_base_CDS.columns = new_column_names
df_base_CDS.to_csv('rrf_CDS.csv') 
df_base_CDS = pd.read_csv('rrf_CDS.csv')
df_base_CDS = df_base_CDS.loc[:,~df_base_CDS.columns.str.contains('^Unnamed')]
print len(df_base_CDS.columns)
df_base_CDS

129


,gene,ks5_reads,ks5_rpc,ks5_rpkm,ks6_reads,ks6_rpc,ks6_rpkm,ks7_reads,ks7_rpc,ks7_rpkm,...,TE_ks15/ks21,TE_ks16/ks22,strand,gene_type,gene_start,gene_end,TU,TU_start,TU_end,MorP
0,aaaD,1.0,0.009709,0.618022,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.276398,+,CDS,580577,580885,NaN,NaN,NaN,NaN
1,aaaE,10.0,0.026455,1.684027,5.0,0.013228,1.580083,12.0,0.031746,3.408674,...,0.654585,0.898292,+,CDS,1203627,1204760,NaN,NaN,NaN,NaN
2,aaeA,27.0,0.086817,5.526426,19.0,0.061093,7.297850,30.0,0.096463,10.357547,...,3.413192,0.715028,-,CDS,3386216,3387148,NaN,NaN,NaN,NaN
3,aaeB,21.0,0.032012,2.037776,8.0,0.012195,1.456759,9.0,0.013720,1.473108,...,0.196868,0.178402,-,CDS,3384243,3386210,NaN,NaN,NaN,NaN
4,aaeR,130.0,0.419355,26.694551,79.0,0.254839,30.441573,106.0,0.341935,36.714719,...,0.712714,0.555354,+,CDS,3387542,3388471,aaeR,3386966.0,3388520.0,Mono
5,aaeX,5.0,0.073529,4.680606,3.0,0.044118,5.270042,1.0,0.014706,1.579018,...,0.374048,inf,-,CDS,3387156,3387359,NaN,NaN,NaN,NaN
6,aas,277.0,0.384722,24.489969,133.0,0.184722,22.065859,136.0,0.188889,20.281611,...,1.192911,0.887469,-,CDS,2971877,2974036,aas,2970724.0,2974071.0,Mono
7,aat,95.0,0.404255,25.733372,42.0,0.178723,21.349275,50.0,0.212766,22.845369,...,0.748097,0.380047,-,CDS,925951,926655,cydD-cydC-aat-infA-serW,925098.0,930220.0,Poly
8,abgA,2.0,0.004577,0.291333,2.0,0.004577,0.546701,6.0,0.013730,1.474232,...,1.028633,0.783126,-,CDS,1401279,1402589,NaN,NaN,NaN,NaN
9,abgB,6.0,0.012448,0.792401,3.0,0.006224,0.743491,2.0,0.004149,0.445532,...,0.187024,0.414596,-,CDS,1399834,1401279,NaN,NaN,NaN,NaN


# csv with all genes

In [11]:
df_base.to_csv('rrf_gene.csv') 
df_base = pd.read_csv('rrf_gene.csv')
df_base = df_base.loc[:,~df_base.columns.str.contains('^Unnamed')]

In [12]:
new_column_names = []
for cn in df_base.columns:
    new_cn=cn
    if cn[0]=='(':
        cn_list = cn.split(',')
        new_cn = cn_list[0][2:-1]+'_'+cn_list[1][2:-2]
    if new_cn[-1]=='_':
        new_cn = new_cn[:-1]
    new_column_names.append(new_cn)
print len(new_column_names)

129


In [13]:
df_base.columns = new_column_names
df_base.to_csv('rrf_gene.csv') 
df_base = pd.read_csv('rrf_gene.csv')
df_base = df_base.loc[:,~df_base.columns.str.contains('^Unnamed')]
print len(df_base.columns)
df_base

129


,gene,ks5_reads,ks5_rpc,ks5_rpkm,ks6_reads,ks6_rpc,ks6_rpkm,ks7_reads,ks7_rpc,ks7_rpkm,...,TE_ks15/ks21,TE_ks16/ks22,strand,gene_type,gene_start,gene_end,TU,TU_start,TU_end,MorP
0,aaaD,1.0,0.009709,0.618022,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.276398,+,CDS,580577,580885,NaN,NaN,NaN,NaN
1,aaaE,10.0,0.026455,1.684027,5.0,0.013228,1.580083,12.0,0.031746,3.408674,...,0.654585,0.898292,+,CDS,1203627,1204760,NaN,NaN,NaN,NaN
2,aaeA,27.0,0.086817,5.526426,19.0,0.061093,7.297850,30.0,0.096463,10.357547,...,3.413192,0.715028,-,CDS,3386216,3387148,NaN,NaN,NaN,NaN
3,aaeB,21.0,0.032012,2.037776,8.0,0.012195,1.456759,9.0,0.013720,1.473108,...,0.196868,0.178402,-,CDS,3384243,3386210,NaN,NaN,NaN,NaN
4,aaeR,130.0,0.419355,26.694551,79.0,0.254839,30.441573,106.0,0.341935,36.714719,...,0.712714,0.555354,+,CDS,3387542,3388471,aaeR,3386966.0,3388520.0,Mono
5,aaeX,5.0,0.073529,4.680606,3.0,0.044118,5.270042,1.0,0.014706,1.579018,...,0.374048,inf,-,CDS,3387156,3387359,NaN,NaN,NaN,NaN
6,aas,277.0,0.384722,24.489969,133.0,0.184722,22.065859,136.0,0.188889,20.281611,...,1.192911,0.887469,-,CDS,2971877,2974036,aas,2970724.0,2974071.0,Mono
7,aat,95.0,0.404255,25.733372,42.0,0.178723,21.349275,50.0,0.212766,22.845369,...,0.748097,0.380047,-,CDS,925951,926655,cydD-cydC-aat-infA-serW,925098.0,930220.0,Poly
8,abgA,2.0,0.004577,0.291333,2.0,0.004577,0.546701,6.0,0.013730,1.474232,...,1.028633,0.783126,-,CDS,1401279,1402589,NaN,NaN,NaN,NaN
9,abgB,6.0,0.012448,0.792401,3.0,0.006224,0.743491,2.0,0.004149,0.445532,...,0.187024,0.414596,-,CDS,1399834,1401279,NaN,NaN,NaN,NaN
